In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
import math
import random

In [2]:
def phi(x, r):
    return r * x * (1 - x)

def get_window(r):
    return ((r - 1) / (2 * r), (r + 1) / (2 * r))

In [3]:
def simple_iterations(epsilon, r):
    (left_border, right_border) = get_window(r)
    iterations = 0
    
    cur_x = random.uniform(left_border, right_border)
    next_x = phi(cur_x, r) 
    
    while (abs(cur_x - next_x) > epsilon):
        cur_x = next_x
        next_x = phi(cur_x, r)
        iterations += 1
        if (iterations > 1000):
            return (-100, 1000)
        
    return (next_x, iterations)

In [4]:
roots = []
roots.append(0)
roots.append(0)

for i in range(1, 10**4):
    cur_r = random.uniform(0.01, 0.99)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    x1 = 0.0
    x2 = 1 - (1 / cur_r)
    if (answer > -50):
        if (abs(answer - x1) < abs(answer - x2) and abs(answer - x1) < 1e-6):
            roots[0] += 1
        elif (abs(answer - x2) < 1e-6):
            roots[1] += 1
print(roots)

[9999, 0]


In [5]:
roots = []
roots.append(0)
roots.append(0)

undefined_roots = []

for i in range(1, 10**4):
    cur_r = random.uniform(1.01, 2.99)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    x1 = 0.0
    x2 = 1 - (1 / cur_r)
    if (answer > -50):
        if (abs(answer - x1) < abs(answer - x2) and abs(answer - x1) < 1e-6):
            roots[0] += 1
        elif (abs(answer - x2) < 1e-6):
            roots[1] += 1
    else:
        undefined_roots.append(cur_r)
print(roots) 
print(undefined_roots)

[0, 9969]
[2.988960160494271, 2.9863340307315553, 2.98974874211265, 2.9869293551021867, 2.984471728313693, 2.987285770559456, 2.984587436664672, 2.9869421585747125, 2.985807817119656, 2.98650768312027, 2.9891279778054853, 2.98828188316665, 2.985375753165715, 2.9886682261139224, 2.989390604103881, 2.9899441717278297, 2.9892306033506895, 2.9865963094335672, 2.989534078768523, 2.9878872129813026, 2.9884802995646855, 2.9869369601719, 2.9845544121731375, 2.9850335970883277, 2.989636417476947, 2.9881969600263756, 2.9892491532313445, 2.9880488033820045, 2.9852377817069016, 2.98894880307831]


In [6]:
count = 0

for i in range(1, 10**4):
    cur_r = random.uniform(3.01, 10.0)
    (answer, iters) = simple_iterations(1e-8, cur_r)
    if (answer < -50):
        count += 1
print(count)

9999


x = r * x * (1 - x)

phi(x) = r * x * (1 - x)

phi'(x) = r - 2rx

чтобы решение сходилось к корню, первое приближение должно лежать в окрестности корня, удовлетворяющей условию

|phi'(x)| <= 1

|phi'(x)| <= 1 => max(0.0, (r - 1) / (2 * r)) < x < (r + 1) / (2 * r)

Если r > 1, то 0 в этой области не содержится, и сходиться к нему не будет

x0 = 0

x1 = 1 - 1/r

Рассмотрим phi'(x0) = r

Чтобы в окрестности корня выполнялось |phi'(x)| < 1, должно быть |r| < 1 => r < 1

Значит, при 0 < r < 1 сходимость к x0 = 0 возможна

Рассмотрим phi'(x1) = 2 - r

|phi'(x)| < 1 => |2 - r| < 1 => 1 < r < 3

При 1 < r < 3 сходимость к x1 = 1 - 1/r возможна

Заметим, что области сходимости к x0 (0; 1) и к x1 (1; 3) не пересекаются

In [7]:
def simple_iterations_with_dir_detection(epsilon, r):
    (left_border, right_border) = get_window(r)
    points = []
    
    dir_changed = 0
    dir_saved = 0
        
    cur_x = random.uniform(left_border, right_border)
    next_x = phi(cur_x, r) 
    points.append(cur_x)
    points.append(next_x)
    iterations = 2
    
    while (abs(cur_x - next_x) > epsilon):
        cur_x = next_x
        next_x = phi(cur_x, r)
        points.append(next_x)
        iterations += 1
        if (iterations > 1000):
            return (-100, -100, -100)
        
    if (abs(next_x) > 1e-6 and abs(next_x - (1 - 1/r)) > 1e-6):
        return (-100, -100, next_x)
    root = next_x
    for i in range (0, len(points) - 1):
        cur_point = points[i]
        next_point = points[i + 1]
        d = (cur_point - root) * (next_point - root)
        if d > 0:
            dir_saved += 1
        elif d < 0:
            dir_changed += 1
            
    return (dir_saved, dir_changed, root)

In [8]:
total_dir_saves = 0
total_dir_changes = 0

errors = []

for i in range(1, 10**4):
    cur_r = random.uniform(0.01, 0.99)
    (dir_saved, dir_changed, answer) = simple_iterations_with_dir_detection(1e-8, cur_r)
    if (abs(answer) < 1e-6):
        total_dir_saves += dir_saved
        total_dir_changes += dir_changed
    else:
        errors.append((cur_r, answer))

print(total_dir_saves, total_dir_changes)
print(errors)

528113 2539
[]


In [9]:
total_dir_saves = 0
total_dir_changes = 0

errors = []

for i in range(1, 10**4):
    cur_r = random.uniform(1.01, 1.99)
    (dir_saved, dir_changed, answer) = simple_iterations_with_dir_detection(1e-8, cur_r)
    if (abs(answer - (1 - 1 / cur_r)) < 1e-6):
        total_dir_saves += dir_saved
        total_dir_changes += dir_changed
    else:
        errors.append((cur_r, answer))

print(total_dir_saves, total_dir_changes)
print(errors)

507241 2481
[]


In [10]:
total_dir_saves = 0
total_dir_changes = 0

errors = []

for i in range(1, 10**4):
    cur_r = random.uniform(2.01, 2.99)
    (dir_saved, dir_changed, answer) = simple_iterations_with_dir_detection(1e-8, cur_r)
    if (abs(answer - (1 - 1 / cur_r)) < 1e-6):
        total_dir_saves += dir_saved
        total_dir_changes += dir_changed
    else:
                errors.append((cur_r, answer))

print(total_dir_saves, total_dir_changes)
print(errors)

3327 553223
[(2.9849359580236428, -100), (2.9857703523221697, -100), (2.9850498040728235, -100), (2.9856967477186296, -100), (2.989472980959873, -100), (2.9899340612746497, -100), (2.987413567925448, -100), (2.989208764124913, -100), (2.9846739872118735, -100), (2.986349300886608, -100), (2.9868054789761125, -100), (2.9844092032387843, -100), (2.987437301341629, -100), (2.985820358025607, -100), (2.9891930657131227, -100), (2.9891912696048415, -100), (2.9863598990469757, -100), (2.989260801528649, -100), (2.9887296930031697, -100), (2.9847341189240133, -100), (2.985298716622744, -100), (2.989740044222911, -100), (2.985365280511323, -100), (2.989337434545777, -100), (2.9856754933517378, -100), (2.984472558990903, -100), (2.989492336388679, -100), (2.987172556279704, -100), (2.986480757592478, -100), (2.9889942248947867, -100), (2.986105387783484, -100), (2.9889725525103015, -100), (2.986021197839344, -100), (2.9891507122311487, -100)]


При 0 < r < 1 сходится в точку x0 = 0.

phi'(x0) = r => 0 < phi' < 1 => монотонная сходимость

При 1 < r < 2 сходится в точку x1 = 1 - 1/r

phi'(x1) = 2 - r => 0 < phi' < 1 => монотонная сходимость

При 2 < r < 3 сходится в точку x1 = 1 - 1/r

phi'(x1) = 2 - r => -1 < phi' < 0 => колебательная сходимость